<h3> Importy

In [147]:
import torch
import torch.nn as nn
import torch.optim as optim
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from random import randint

<h3>Wczytanie i przygotowanie danych

In [148]:
# Parametry danych
vocab_size = 10000  # Liczba najczęściej występujących słów
max_len = 128       # Maksymalna długość sekwencji (przycinanie/padowanie)

# Wczytanie danych IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Padowanie sekwencji
x_train = pad_sequences(x_train, maxlen=max_len, padding="post", truncating="post")
x_test = pad_sequences(x_test, maxlen=max_len, padding="post", truncating="post")

# Konwersja danych na tensory PyTorch
x_train, y_train = torch.tensor(x_train), torch.tensor(y_train)
x_test, y_test = torch.tensor(x_test), torch.tensor(y_test)

In [149]:
number = randint(1, len(x_train) + 1)
print(x_train.shape)
print(len(x_train[number]))
print(max(x_train[number]))
print(x_train[number])

torch.Size([25000, 128])
128
tensor(8685, dtype=torch.int32)
tensor([   1, 7619, 1160,    4,  801,  114,    7,    4,  402, 1355, 1179, 5351,
         184,  185,  539,   11, 7623,   13,   28,    8,  202,   12,   56,   18,
           4, 1723,   37,  343,    6, 4411,   18,   49,    7,    4,  402, 1320,
        1189,  665,   25,  165,  104,   18,    6,  333,  225,  170,    8,   30,
         489,   11,    4,  365,  149, 7619,   16,   40,  319,   35, 2374,  116,
         707,  140,  143,   45, 2272,   56,   49,    7,    4,   91, 2094,  913,
        3990, 5585,  139,   26,  256,   46,   19,    4, 8685,    7,    6,    2,
           2,   13, 3550,  138,   36,  161,   43,  276,    4, 5435, 4320,   23,
         370,   38,    4, 1507,  586,   28,    8, 1343, 6787,   68,    2,    4,
         107,  293,  539,   71,  540, 2218,   18,   68,  830,    6, 2367,  247,
          74,  676, 3779,    2,   17,    4,  293, 2284], dtype=torch.int32)


<h3>Budowa modelu

In [150]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, rnn_type):
        super(SentimentRNN, self).__init__()
        self.rnn_type = rnn_type
        self.embedding = nn.Embedding(vocab_size, embed_dim)    # embedding slów
        self.fc = nn.Linear(hidden_dim, output_dim)             # wartswa liniowa
        self.activation = nn.Sigmoid()                          # funkcja aktywacji

        if self.rnn_type == "LSTM":                             # warstwa rekurencyjna
            self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        elif self.rnn_type == "RNN":
            self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        else:
            raise ValueError("Unsupported RNN type. Choose 'LSTM' or 'RNN'.")

    # RNN zwraca tylko dwa elementy: sekwencję wyjść i ostatni stan ukryty. W przypadku RNN nie występuje dodatkowy cel ukryty, jak w LSTM
    def forward(self, x):
        x = self.embedding(x)
        if self.rnn_type == "LSTM": _, (hidden, _) = self.rnn(x)
        elif self.rnn_type == "RNN": _, hidden = self.rnn(x)
        else: raise ValueError("Unknown RNN type in forward method. Choose 'LSTM' or 'RNN'.")
        x = self.fc(hidden[-1])
        return self.activation(x)  # Funkcja aktywacji (sigmoid)

# Parametry modelu
embed_dim = 64
hidden_dim = 128
output_dim = 1      # klasyfikacja binarna

models = [SentimentRNN(vocab_size, embed_dim, hidden_dim, output_dim, rnn_type="RNN"),
          SentimentRNN(vocab_size, embed_dim, hidden_dim, output_dim, rnn_type="LSTM")]

<h3>Implementacja funkcji treningu i ewaluacji

In [151]:
batch_size = 64

# Dataloader
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Funkcja treningu
def train_model(model, train_loader, criterion = nn.BCELoss(), epochs = 2, learning_rate = 0.001):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        total_loss = 0
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            predictions = model(x_batch).squeeze()
            loss = criterion(predictions, y_batch.float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        acc = evaluate_model(model, x_test, y_test)
        print(f"Model {model.rnn_type}, epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}, Acc: {acc * 100:.2f}%")

# Funkcja do ewaluacji
def evaluate_model(model, x_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(x_test).squeeze()
        predictions = torch.round(predictions)
        accuracy = (predictions == y_test).float().mean().item()
    return accuracy

<h3>Trening i ocena modelu

In [152]:
for model in models:
    train_model(model, train_loader)

Model RNN, epoch 1, Loss: 0.6963, Acc: 51.96%
Model RNN, epoch 2, Loss: 0.6908, Acc: 51.30%
Model LSTM, epoch 1, Loss: 0.6860, Acc: 54.11%
Model LSTM, epoch 2, Loss: 0.6810, Acc: 51.90%
